In [ ]:
# processed data csv path
# processed_tweets_path = '/content/drive/MyDrive/sentiment analysis files/tokenized_data_joined.csv'

In [ ]:
# import pandas as pd

In [ ]:
# load the processed data from the csv file
# data = pd.read_csv(processed_tweets_path).iloc[:, 1:]
# data.head()

,target,text
0,0,twitpic com zl awww bummer shoulda get david c...
1,0,upset update facebook texting cry result schoo...
2,0,manage save rest bound
3,0,body feel itchy like fire
4,0,behave mad


#Actual code starts here with the pickle file

In [ ]:
"""
# pickling a dataframe to a pickle file
import pickle
path = ""
df = pd.DataFrame()

with open(path) as file_handle:
  pickle.dump(df, file_handle)
"""

'\n# pickling a dataframe to a pickle file\nimport pickle\npath = ""\ndf = pd.DataFrame()\n\nwith open(path) as file_handle:\n  pickle.dump(df, file_handle)\n'

In [ ]:
!cp /content/drive/MyDrive/tokenized_data.pkl /content

In [ ]:
!ls

drive  sample_data  tokenized_data.pkl


In [5]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from keras.layers import TextVectorization
import numpy as np
import pickle

In [ ]:
# with open("/content/drive/MyDrive/tokenized_data.pkl") as f:
reduced_data = pd.read_pickle('/content/tokenized_data.pkl')

In [ ]:
reduced_data.head()

,target,text
0,0,"[twitpic, com, zl, awww, bummer, shoulda, get,..."
1,0,"[upset, update, facebook, texting, cry, result..."
2,0,"[manage, save, rest, bound]"
3,0,"[body, feel, itchy, like, fire]"
4,0,"[behave, mad]"


##**STEP - 1:** Find the length of the longest list in the text column

In [ ]:
# find the maximum sequence length using ordinary for loop
max_len = 0
for text in reduced_data["text"]:
  temp = len(text)
  if temp > max_len:
    max_len = temp
print(max_len)

50


In [ ]:
# find the maximum sequence length using apply function 
max_len = max(reduced_data["text"].apply(lambda x: len(x)))
print(max_len)

50


In [ ]:
# find the maximum sequence length using map function, **turns out to be very slow**
# with ThreadPoolExecutor(max_workers=4) as pool:
#   max_len = max(list(pool.map(lambda x: len(x), list(reduced_data["text"]))))
# print(max_len)

In [ ]:
def filter_by_len(token_list):
  if len(token_list) >= 10:
    return True
  return False

In [ ]:
reduced_data["text"]

0          [twitpic, com, zl, awww, bummer, shoulda, get,...
1          [upset, update, facebook, texting, cry, result...
2                                [manage, save, rest, bound]
3                            [body, feel, itchy, like, fire]
4                                              [behave, mad]
                                 ...                        
1599995                        [woke, school, best, feeling]
1599996      [thewdb, com, cool, hear, old, walt, interview]
1599997                 [ready, mojo, makeover, ask, detail]
1599998    [happy, th, birthday, boo, alll, time, tupac, ...
1599999    [happy, charitytuesday, thenspcc, sparkscharit...
Name: text, Length: 1600000, dtype: object

##**STEP-2:** Remove the tweets having less than 10 tokens, to reduce network training time

In [ ]:
# reduced_data["text"] = list(filter(filter_by_len, list(reduced_data["text"])))
filtered_reduced_data = reduced_data[reduced_data["text"].apply(lambda x: len(x)) >= 10]

In [ ]:
filtered_reduced_data.shape

(214193, 2)

###**WORD EMBEDDING:** Computer stores the meaning of a word in form of vectors called Word Embeddings.

In [ ]:
filtered_reduced_data

,target,text
0,0,"[twitpic, com, zl, awww, bummer, shoulda, get,..."
16,0,"[hollis, death, scene, hurt, severely, watch, ..."
35,0,"[ok, sick, spend, hour, sit, shower, sick, sta..."
39,0,"[bed, class, work, gym, class, day, go, fly, m..."
44,0,"[fall, asleep, hear, tracy, girl, body, find, ..."
...,...,...
1599954,4,"[god, work, mysteriously, learn, think, world,..."
1599965,4,"[thanks, follow, nice, website, check, carriag..."
1599966,4,"[get, home, hour, ago, ate, lunch, watch, tv, ..."
1599985,4,"[lead, singer, band, beware, fall, prey, lsd, ..."


In [ ]:
filtered_reduced_data["text"] = filtered_reduced_data["text"].apply(lambda x: " ".join(x))

<ipython-input-113-e43d4864066f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reduced_data["text"] = filtered_reduced_data["text"].apply(lambda x: " ".join(x))


In [ ]:
filtered_reduced_data.head()

,target,text
0,0,twitpic com zl awww bummer shoulda get david c...
16,0,hollis death scene hurt severely watch film wr...
35,0,ok sick spend hour sit shower sick stand hold ...
39,0,bed class work gym class day go fly miss girlf...
44,0,fall asleep hear tracy girl body find sad hear...


In [ ]:
max_vocab_size = 30000
embedding_dim = 64    # the size of word embedding to be used

In [ ]:
vectorization_layer = TextVectorization(max_tokens=max_vocab_size, output_sequence_length=max_len)

In [ ]:
vectorization_layer.adapt(data=np.array(filtered_reduced_data["text"]))

In [ ]:
pickle.dump({'config': vectorization_layer.get_config(),
             'weights': vectorization_layer.get_weights()}
            , open("/content/token_integer_mapping.pkl", "wb"))

In [ ]:
# !mv /content/token_integer_mapping.pkl /content/drive/MyDrive

##**START FROM HERE**

In [1]:
!cp /content/drive/MyDrive/token_integer_mapping.pkl /content

In [2]:
!ls

drive  sample_data  token_integer_mapping.pkl


In [6]:
with open("/content/token_integer_mapping.pkl", "rb") as file_handle:
  loaded_vectorization_layer = pickle.load(file_handle)

In [7]:
loaded_vectorization_layer

{'config': {'name': 'text_vectorization',
  'trainable': True,
  'dtype': 'string',
  'batch_input_shape': (None,),
  'max_tokens': 30000,
  'standardize': 'lower_and_strip_punctuation',
  'split': 'whitespace',
  'ngrams': None,
  'output_mode': 'int',
  'output_sequence_length': 50,
  'pad_to_max_tokens': False,
  'sparse': False,
  'ragged': False,
  'vocabulary': None,
  'idf_weights': None,
  'encoding': 'utf-8',
  'vocabulary_size': 30000},
 'weights': [array([b'quot', b'go', b'day', ..., b'fabi', b'fabb', b'faar'],
        dtype=object)]}